Import

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle


Utilities

In [ ]:
def save_with_pickle(compressed_input,filename):
    pickle_out = open(filename+"_compressed","wb")
    pickle.dump(compressed_input,pickle_out)


def get_with_pickle(filename):
    compressed_image = pickle.load(open(filename, "rb"))
    print(filename)
    return compressed_image

LZW Encodeur

In [ ]:
def LZW_Encode(pixels_input):
    
    DICT_SIZE = 256
    SET_INPUT = set(pixels_input)
    DICTIONNARY_LZW = {}

    encoded_image = []
    
    for pixels in SET_INPUT:
        DICTIONNARY_LZW[str(pixels).zfill(3)]=pixels

    word = ""

    for pixels in pixels_input:
        pixels = str(pixels).zfill(3) # on convertit le pixels en str pour le concatener
        word_k = word+pixels # wk = w+k

        # --- si la sequence est dans le dictionnaire on ajoute conitnue de chercher une sequence + grande
        if word_k in DICTIONNARY_LZW:
            word = word_k

        # --- sinon on ajoute l'ancienne sequence et on ajoute la sequence courante au dico
        else:
            encoded_image.append(DICTIONNARY_LZW[word])
            DICTIONNARY_LZW[word_k]=DICT_SIZE
            DICT_SIZE+=1
            word = pixels
            
    # --- on verifie qu'il reste plus aucune sequence a ajouter
    if word != "":
        encoded_image.append(DICTIONNARY_LZW[word])
    
    return encoded_image

LZW Decodeur

In [ ]:
def LZW_Decode(encoded_pixels,width,height):

    DICT_SIZE = 256
    DICTIONNARY_LZW = {}
    decoded_pixels = []

    # --- on crée un dicitionnaire de toutes les valeurs de pixels possible avec comme valeur
    # --- ecrit sur 3 bits
    for i in range(DICT_SIZE):
        DICTIONNARY_LZW[str(i)]=str(i).zfill(3)


    # --- on prend la valeur du premier caractere encodé
    previous = str(encoded_pixels[0]).zfill(3)
    encoded_pixels  = encoded_pixels[1:]

    # --- on le decode
    decoded_pixels.append(previous)

    for pixels in encoded_pixels:

        pixels = str(pixels) # --- on convertit le pixels en string
        pre_output = ""

        if pixels in DICTIONNARY_LZW: # --- si le pixel est dans le dico on decode sa valeur
            pre_output = DICTIONNARY_LZW[pixels]
        else :
            pre_output = previous.zfill(3)+previous[:3] # --- sinon on suppose que ça valeur égale a ça

        # --- on ajoute la valeur decodé au message
        decoded_pixels.append(pre_output)
        # --- on ajoute l'ancienne sequence et le premier pixel de la nouvelle
        DICTIONNARY_LZW[str(DICT_SIZE)] = previous.zfill(3)+pre_output[:3]
        DICT_SIZE += 1
        previous = pre_output
    
    decoded_image = []

    # --- on parcours les valeurs décodé on on les decomposes sur 3 caracteres pour les convertirs ensuite
    for value in decoded_pixels:
        for i in range(0,len(value),3):
            decoded_image.append(value[i:i+3])


    decoded_image = [int(x) for x in decoded_image]

    image = np.array(decoded_image)
    print(len(image))

    image = image.reshape(width,height)

    return image

Main

In [ ]:
def MainLZW():
    liste_images = [
        "image_1.bmp",
        "image_2.bmp",
        "image_3.bmp",

        "binary_random1.png",
        "binary_cablecar1.bmp",

        "image_4.png", # en niveau de gris
        "cablecar.bmp" # en niveau de gris
    ]

    for files in liste_images:

        image_tp_4 = cv2.imread(files, 0)
        width,height = image_tp_4.shape

        print("\n----------- Image -----------\n")
        plt.imshow(image_tp_4)
        plt.show()
        print("----------- ***** -----------\n")

        # --- Applatir l'image
        string_image = image_tp_4.flatten().tolist()
        # --- Image compressé
        compressed_image = LZW_Encode(string_image)

        # --- Sauvegarder l'image avec Pickle
        save_with_pickle(compressed_image,files)
        # --- Taux de compression

        compression_rate = (1-(len(compressed_image)/2)/len(string_image))*100
        print(f"Taux de compression : {compression_rate}")

        compressed_image = get_with_pickle(f"{files}_compressed")
        decompressed_image = LZW_Decode(compressed_image,int(width),int(height))

        print("----------- Image-Décodé -----------\n")
        plt.imshow(decompressed_image)
        plt.show()
        print("----------- ************ -----------\n")

        # ---- on calcule la différence
        var = np.count_nonzero(image_tp_4-decompressed_image)
        print(f"Soustraction des deux images : \n{var}")

Programme

In [ ]:
MainLZW()